In [48]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from pathlib import Path
from vectorization.v_a_c import *

In [49]:
API_KEY = "MjQxYzdhYjQtZjk5OC00ZjUzLWJjZTYtZGMyOGJhNGQ1OGFmOmUzNTA0Mjg4LWQ5NGEtNDdhNy1hNDkzLTgwNjczZTFjMGE1YQ=="

In [50]:
CODE_DIR = Path("..\\data\\code_prod")
DOC_DIR = Path("..\\data\\documentation_prod")
VECTOR_STORE_PATH = "langchain_vector_store"

In [51]:
# CODE_DIR = Path("..\\data\\code")
# DOC_DIR = Path("..\\data\\documentation")
# VECTOR_STORE_PATH = "langchain_vector_store_test"

In [52]:
# model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': False}
# embeddings = HuggingFaceEmbeddings(model_name=model_name,
#                                   model_kwargs=model_kwargs,
#                                   encode_kwargs=encode_kwargs)

In [53]:
embeddings=GigaChatEmbeddings(
        credentials=API_KEY,
        scope="GIGACHAT_API_PERS",
        verify_ssl_certs=False,
)

In [54]:
system = SmartCodeDocSystem(embeddings, chunk_size=600, chunk_overlap=100)

In [55]:
documents = system.load_and_process_files(CODE_DIR, DOC_DIR)

system.create_vector_store(VECTOR_STORE_PATH)

Загрузка и обработка файлов...
Обработка кода из ..\data\code_prod
  Обработано 10/1586 файлов кода
  Обработано 20/1586 файлов кода
  Обработано 30/1586 файлов кода
  Обработано 40/1586 файлов кода
  Обработано 50/1586 файлов кода
  Обработано 60/1586 файлов кода
  Обработано 70/1586 файлов кода
  Обработано 80/1586 файлов кода
  Обработано 90/1586 файлов кода
  Обработано 100/1586 файлов кода
  Обработано 110/1586 файлов кода
  Обработано 120/1586 файлов кода
  Обработано 130/1586 файлов кода
  Обработано 140/1586 файлов кода
  Обработано 150/1586 файлов кода
  Обработано 160/1586 файлов кода
  Обработано 170/1586 файлов кода
  Обработано 180/1586 файлов кода
  Обработано 190/1586 файлов кода
  Обработано 200/1586 файлов кода
  Обработано 210/1586 файлов кода
  Обработано 220/1586 файлов кода
  Обработано 230/1586 файлов кода
  Обработано 240/1586 файлов кода
  Обработано 250/1586 файлов кода
  Обработано 260/1586 файлов кода
  Обработано 270/1586 файлов кода
  Обработано 280/1586 фа

In [56]:
system.load_vector_store(VECTOR_STORE_PATH)

Загрузка векторного хранилища из langchain_vector_store
Векторное хранилище загружено


True

In [57]:
llm = GigaChat(
    credentials=API_KEY,
    verify_ssl_certs=False,
)

smart_retriever = SmartRetriever(smart_system=system, k=3)

prompt = create_smart_prompt()
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
retrieval_chain = create_retrieval_chain(smart_retriever, document_chain)

In [58]:
query = "How to use FAISS vectorstore in LangChain?"

try:
    response = retrieval_chain.invoke({"input": query})
    
    print("ОТВЕТ:")
    print(response["answer"])
    
    search_result = system.smart_search(query, k=3)
    print(f"\nИНФО О ПОИСКЕ:")
    print(f"Тип поиска: {search_result.search_type}")
    print(f"Найдено документов: {len(search_result.documents)}")
    
    code_count = len([d for d in search_result.documents if d.metadata.get('type') == 'code'])
    doc_count = len([d for d in search_result.documents if d.metadata.get('type') == 'doc'])
    print(f"Код: {code_count}, Документация: {doc_count}")
    
except Exception as e:
    print(f"Ошибка: {e}")

ОТВЕТ:
### Как использовать FAISS векторный магазин в LangChain?

В библиотеке LangChain можно использовать векторные магазины для хранения и управления векторами слов, которые представляют собой представления слов в пространстве многомерных векторов. Одним из популярных библиотек для индексации и поиска подобных векторов является **FAISS** (Facebook AI Similarity Search). В данном контексте мы рассмотрим, как создать экземпляр FAISS векторного магазина и интегрировать его с LangChain для выполнения задач поиска.

#### Пример использования FAISS в LangChain

Для начала вам потребуется установить необходимые зависимости:

```bash
%pip install --upgrade --quiet  langchain-openai faiss-cpu  
```

Затем можно приступить к созданию экземпляра FAISS векторного магазина.

```python
from langchain_community.vectorstores import FAISS

# Создание FAISS векторного магазина
faiiss = FAISS()

# Добавление данных в векторный магазин
documents = ["This is a test document", "Another test document"]
ve